# Accessing Pickled Metrics And Prediction Probabilities - Testing Data- (for STREAMLINE)
The pipeline outputs pickled objects with all the metric results, elements needed to build the ROC and PRC plots, as well as the prediction probabilities on the testing data across all datasets, algorithm models, and CV dataset partitions. 

This notebook illustrates how the user can access the pickled metric information saved as a list object. 

It includes (1) grabbing and calculating all average metric scores over the CV partitions, (2) grabbing the elements needed to build the average ROC plot, (3) grabbing the elementes needed to build the average PRC plot, (4) grabbing and reporting average model feature importance scores, and (5) grabbing and reporting the model testing prediction probabilities for each instance of the dataset. 

When run, this last item will generate a new folder in the pipeline's output experiment folder in the 'model_evaluation' folder for each dataset. Here the (case/i.e. code 1) prediction probabilities are reported as a .csv file for each algorithm and CV partition pair.  In these files is the instance's true outcome value, the unique instance ID, and the predicted probability of the instance being case/code 1. 
 

## Import Packages

In [1]:
import os
import pandas as pd
import pickle
import numpy as np
from statistics import mean
from scipy import interp,stats
import warnings
warnings.filterwarnings('ignore')

# Jupyter Notebook Hack: This code ensures that the results of multiple commands within a given cell are all displayed, rather than just the last. 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Set Run Parameters

In [2]:
experiment_path = "../DemoOutput/demo_experiment"
target_data_list = None # None if user wants to generate visualizations for all analyzed datasets otherwise list of datasets to run
algorithms = [] # use empty list if user wishes re-evaluate all modeling algorithms that were run in pipeline.

## Automatically Detect Dataset Names

In [3]:
# Get dataset paths for all completed dataset analyses in experiment folder
datasets = os.listdir(experiment_path)

# Name of experiment folder
experiment_name = experiment_path.split('/')[-1] 

datasets = os.listdir(experiment_path)
remove_list = ['.DS_Store', 'metadata.pickle', 'metadata.csv', 'algInfo.pickle',
                'DatasetComparisons', 'jobs', 'jobsCompleted', 'logs',
                'KeyFileCopy', 'dask_logs',
                experiment_name + '_ML_Pipeline_Report.pdf']
for text in remove_list:
    if text in datasets:
        datasets.remove(text)

datasets = sorted(datasets) # ensures consistent ordering of datasets
print("Analyzed Datasets: " + str(datasets))

Analyzed Datasets: ['hcc-data_example', 'hcc-data_example_custom']


## Load Other Necessary Parameters

In [4]:
# Unpickle metadata from previous phase
file = open(experiment_path + '/' + "metadata.pickle", 'rb')
metadata = pickle.load(file)
file.close()
# Load variables specified earlier in the pipeline from metadata
class_label = metadata['Class Label']
instance_label = metadata['Instance Label']
cv_partitions = int(metadata['CV Partitions'])

# Unpickle algorithm information from previous phase
file = open(experiment_path + '/' + "algInfo.pickle", 'rb')
algInfo = pickle.load(file)
file.close()
algorithms = []
abbrev = {}
for key in algInfo:
    if algInfo[key][0]: # If that algorithm was used
        algorithms.append(key)
        abbrev[key] = (algInfo[key][1])

print("Algorithms Ran: " + str(algorithms))

Algorithms Ran: ['Logistic Regression', 'Naive Bayes', 'Decision Tree']


## Extract Metric List and Cacluate CV Averages

In [5]:
def print_results(algorithm, full_path):
    # Define evaluation stats variable lists
        s_bac = [] # balanced accuracies
        s_ac = [] # standard accuracies
        s_f1 = [] # F1 scores
        s_re = [] # recall values
        s_sp = [] # specificities
        s_pr = [] # precision values
        s_tp = [] # true positives
        s_tn = [] # true negatives
        s_fp = [] # false positives
        s_fn = [] # false negatives
        s_npv = [] # negative predictive values
        s_lrp = [] # likelihood ratio positive values
        s_lrm = [] # likelihood ratio negative values
        
        aucs = [] #areas under ROC curve
        praucs = [] #area under PRC curve
        aveprecs = [] #average precisions for PRC
        
        for cv_count in range(0, cv_partitions): #loop through cv's
            #Load pickled metric file for given algorithm and cv
            result_file = full_path + '/model_evaluation/pickled_metrics/' + abbrev[algorithm] + "_CV_" + str(cv_count) + "_metrics.pickle"
            file = open(result_file, 'rb')
            results = pickle.load(file)
            file.close()
            
            #Separate pickled results
            metric_list = results[0] #First item in pickled list is the metric list (set of standard classification metrics)
            roc_auc = results[3] #Fourth item is the ROC AUC
            prec_rec_auc = results[6] #Seventh item is the PRC AUC
            ave_prec = results[7] #Eighth item is the average precision of PRC
            
            #Separate metrics from metricList
            s_bac.append(metric_list[0])
            s_ac.append(metric_list[1])
            s_f1.append(metric_list[2])
            s_re.append(metric_list[3])
            s_sp.append(metric_list[4])
            s_pr.append(metric_list[5])
            s_tp.append(metric_list[6])
            s_tn.append(metric_list[7])
            s_fp.append(metric_list[8])
            s_fn.append(metric_list[9])
            s_npv.append(metric_list[10])
            s_lrp.append(metric_list[11])
            s_lrm.append(metric_list[12])
            
            aucs.append(roc_auc)
            praucs.append(prec_rec_auc)
            aveprecs.append(ave_prec)
            
        results = {'Balanced Accuracy': mean(s_bac), 'Accuracy': mean(s_ac), 
                   'F1_Score': mean(s_f1), 'Sensitivity (Recall)': mean(s_re), 
                   'Specificity': mean(s_sp),'Precision (PPV)': mean(s_pr), 
                   'TP': mean(s_tp), 'TN': mean(s_tn), 'FP': mean(s_fp), 
                   'FN': mean(s_fn), 'NPV': mean(s_npv), 'LR+': mean(s_lrp), 
                   'LR-': mean(s_lrm), 'ROC_AUC': mean(aucs),'PRC_AUC': mean(praucs), 
                   'PRC_APS': mean(aveprecs)}
        print(results)

## Extract list of increasing false positive rates and true positive rates for constructing ROC

In [6]:
if target_data_list: # User specified one analyzed dataset above (if more than one were analyzed)
    for each in datasets:
        if not each in target_data_list:
            datasets.remove(each)
print("Vizualized Datasets: " + str(datasets))
for each in datasets: 
    print("---------------------------------------")
    print(each)
    print("---------------------------------------")
    full_path = experiment_path + '/' + each
    for algorithm in algorithms: #loop through algorithms
        print(algorithm)
        print_results(algorithm, full_path)

Vizualized Datasets: ['hcc-data_example', 'hcc-data_example_custom']
---------------------------------------
hcc-data_example
---------------------------------------
Logistic Regression
{'Balanced Accuracy': 0.6862745098039215, 'Accuracy': 0.6909090909090909, 'F1_Score': 0.6236199201315481, 'Sensitivity (Recall)': 0.6666666666666666, 'Specificity': 0.7058823529411765, 'Precision (PPV)': 0.5878592690186893, 'TP': 14, 'TN': 24, 'FP': 10, 'FN': 7, 'NPV': 0.7730429292929293, 'LR+': 2.365147198480532, 'LR-': 0.47630443630443636, 'ROC_AUC': 0.7623716153127917, 'PRC_AUC': 0.6622572580214863, 'PRC_APS': 0.6782941296365633}
Naive Bayes
{'Balanced Accuracy': 0.6386554621848739, 'Accuracy': 0.6545454545454545, 'F1_Score': 0.5474576271186441, 'Sensitivity (Recall)': 0.5714285714285714, 'Specificity': 0.7058823529411764, 'Precision (PPV)': 0.6081871345029239, 'TP': 12, 'TN': 24, 'FP': 10, 'FN': 9, 'NPV': 0.7284503173249977, 'LR+': 2.9829725829725824, 'LR-': 0.6162726368510574, 'ROC_AUC': 0.73832866

In [7]:
if target_data_list: # User specified one analyzed dataset above (if more than one were analyzed)
    for each in datasets:
        if not each in target_data_list:
            datasets.remove(each)
print("Vizualized Datasets: " + str(datasets))

for each in datasets: 
    print("---------------------------------------")
    print(each)
    print("---------------------------------------")
    full_path = experiment_path+ '/' + each
    for algorithm in algorithms: #loop through algorithms
        print(algorithm)
        # Define evaluation stats variable lists
        tprs = [] # true postitive rates
        mean_fpr = np.linspace(0, 1, 100) # used to plot all CVs in single ROC plot
        
        for cv_count in range(0, cv_partitions): #loop through cv's
            # Load pickled metric file for given algorithm and cv =
            result_file = full_path + '/model_evaluation/pickled_metrics/' + abbrev[algorithm] + "_CV_" + str(cv_count) + "_metrics.pickle"
            file = open(result_file, 'rb')
            results = pickle.load(file)
            file.close()
            
            #Separate pickled results
            fpr = results[1]
            tpr = results[2]

            tprs.append(interp(mean_fpr, fpr, tpr))
            tprs[-1][0] = 0.0

        results = {'tprs': np.mean(tprs, axis=0)}
        
        print(results)
        #print('fprs: '+str(mean_fpr))

Vizualized Datasets: ['hcc-data_example', 'hcc-data_example_custom']
---------------------------------------
hcc-data_example
---------------------------------------
Logistic Regression
{'tprs': array([0.        , 0.19047619, 0.19047619, 0.3015873 , 0.3015873 ,
       0.3015873 , 0.36507937, 0.36507937, 0.36507937, 0.3968254 ,
       0.3968254 , 0.3968254 , 0.44444444, 0.44444444, 0.44444444,
       0.49206349, 0.49206349, 0.49206349, 0.50793651, 0.50793651,
       0.50793651, 0.55555556, 0.55555556, 0.55555556, 0.61904762,
       0.61904762, 0.61904762, 0.65079365, 0.65079365, 0.65079365,
       0.66666667, 0.66666667, 0.66666667, 0.74603175, 0.74603175,
       0.76190476, 0.76190476, 0.76190476, 0.79365079, 0.79365079,
       0.79365079, 0.79365079, 0.79365079, 0.79365079, 0.80952381,
       0.80952381, 0.80952381, 0.85714286, 0.85714286, 0.85714286,
       0.9047619 , 0.9047619 , 0.9047619 , 0.9047619 , 0.9047619 ,
       0.9047619 , 0.9047619 , 0.9047619 , 0.9047619 , 0.9047619 ,
 

## Extract list of increasing precision and recall values for constructing PRC

In [8]:
if target_data_list: # User specified one analyzed dataset above (if more than one were analyzed)
    for each in datasets:
        if not each in target_data_list:
            datasets.remove(each)
print("Vizualized Datasets: " + str(datasets))

for each in datasets: 
    print("---------------------------------------")
    print(each)
    print("---------------------------------------")
    full_path = experiment_path + '/' + each
    for algorithm in algorithms: #loop through algorithms
        print(algorithm)
        # Define evaluation stats variable lists
        precs = [] # true postitive rates
        mean_recall = np.linspace(0, 1, 100) # used to plot all CVs in single PRC plot
        
        for cv_count in range(0, cv_partitions): #loop through cv's
            #Load pickled metric file for given algorithm and cv
            result_file = full_path + '/model_evaluation/pickled_metrics/' + abbrev[algorithm] + "_CV_" + str(cv_count) + "_metrics.pickle"
            file = open(result_file, 'rb')
            results = pickle.load(file)
            file.close()
            
            #Separate pickled results
            prec = results[4]
            recall = results[5]

            precs.append(interp(mean_recall, recall, prec))

        results = {'precs': np.mean(precs, axis=0)}

        print(results)
        #print('recall: '+str(mean_recall))

Vizualized Datasets: ['hcc-data_example', 'hcc-data_example_custom']
---------------------------------------
hcc-data_example
---------------------------------------
Logistic Regression
{'precs': array([0.66666667, 0.7020202 , 0.73737374, 0.77272727, 0.80808081,
       0.67340067, 0.6969697 , 0.72053872, 0.74410774, 0.76767677,
       0.78451178, 0.7962963 , 0.80808081, 0.81986532, 0.83164983,
       0.79040404, 0.7986532 , 0.80690236, 0.81515152, 0.72326439,
       0.73055956, 0.73785474, 0.74514991, 0.75244509, 0.7405549 ,
       0.74629411, 0.75203332, 0.75777253, 0.76351174, 0.74052491,
       0.7452085 , 0.74989208, 0.75457567, 0.73333333, 0.73761861,
       0.74190389, 0.74618916, 0.75047444, 0.75458372, 0.75825298,
       0.76192225, 0.76559152, 0.76926079, 0.74251747, 0.74592002,
       0.74932257, 0.75272513, 0.75612768, 0.75918584, 0.76218661,
       0.76518738, 0.76818815, 0.72816776, 0.73140934, 0.73465093,
       0.73789251, 0.74113409, 0.62743298, 0.63131253, 0.63519209,


## Extract Average Model Feature Importance Estimates (Over CVs)

In [9]:
if target_data_list: # User specified one analyzed dataset above (if more than one were analyzed)
    for each in datasets:
        if not each in target_data_list:
            datasets.remove(each)
print("Vizualized Datasets: " + str(datasets))

for each in datasets: 
    print("---------------------------------------")
    print(each)
    print("---------------------------------------")
    full_path = experiment_path + '/' + each
    original_headers = pd.read_csv(full_path + "/exploratory/OriginalFeatureNames.csv", sep=',').columns.values.tolist() # Get Original Headers
    for algorithm in algorithms: #loop through algorithms
        print(algorithm)
        # Define evaluation stats variable lists
        FI_ave = [0] * len(original_headers)  # used to save average FI scores over all cvs. (all original features in dataset prior to feature selection included)
        
        for cv_count in range(0, cv_partitions): # loop through cv's
            #Load pickled metric file for given algorithm and cv
            result_file = full_path + '/model_evaluation/pickled_metrics/' + abbrev[algorithm] + "_CV_" + str(cv_count) + "_metrics.pickle"
            file = open(result_file, 'rb')
            results = pickle.load(file)
            file.close()
            
            #Separate pickled results
            fi = results[8]
            
            # Format feature importance scores as list (takes into account that all features are not in each CV partition)
            tempList = []
            j = 0
            headers = pd.read_csv(full_path + '/CVDatasets/' + each + '_CV_' + str(cv_count) + '_Test.csv').columns.values.tolist()
            if instance_label != None: 
                headers.remove(instance_label)
            headers.remove(class_label)
            for feature in original_headers:
                if feature in headers:  # Check if current feature from original dataset was in the partition
                    # Deal with features not being in original order (find index of current feature list.index()
                    f_index = headers.index(feature)
                    FI_ave[j] += fi[f_index]
                j += 1
            
        #Turn FI sums into averages
        for i in range(0, len(FI_ave)):
            FI_ave[i] = FI_ave[i] / float(cv_partitions)

        fi_dict = {}
        for key in original_headers:
            for value in FI_ave:
                fi_dict[key] = value
                FI_ave.remove(value)
                break  
                
        print(fi_dict)

Vizualized Datasets: ['hcc-data_example', 'hcc-data_example_custom']
---------------------------------------
hcc-data_example
---------------------------------------
Logistic Regression
{'Gender': 0.006384220354808589, 'Symptoms': 0.011939775910364117, 'Alcohol': 0.0, 'Hepatitis B Surface Antigen': 0.00934873949579833, 'Hepatitis B e Antigen': 0.0004901960784313708, 'Hepatitis B Core Antibody': 0.001225490196078427, 'Hepatitis C Virus Antibody': 0.02291083099906628, 'Cirrhosis': -9.337068160597539e-05, 'Endemic Countries': 0.010819327731092454, 'Smoking': 0.009383753501400534, 'Diabetes': 0.05299953314659194, 'Obesity': 0.0005835667600373499, 'Hemochromatosis': 0.0, 'Arterial Hypertension': 0.02274743230625585, 'Chronic Renal Insufficiency': 0.0004901960784313708, 'Human Immunodeficiency Virus': 0.003968253968253954, 'Nonalcoholic Steatohepatitis': 0.0, 'Esophageal Varices': 0.0020541549953314695, 'Splenomegaly': -0.0003267973856209102, 'Portal Hypertension': 0.018218954248366014, 'Por

## Extract and Report Case (i.e. class 1) Prediction Probabilities For all instances in each Testing Dataset

In [10]:
if target_data_list: # User specified one analyzed dataset above (if more than one were analyzed)
    for each in datasets:
        if not each in target_data_list:
            datasets.remove(each)
print("Vizualized Datasets: " + str(datasets))

for each in datasets: 
    print("---------------------------------------")
    print(each)
    print("---------------------------------------")

    full_path = experiment_path + '/' + each
    
    # Make folder in experiment folder/datafolder to store all prediction probabilities per algorithm/CV combination
    if not os.path.exists(full_path + '/model_evaluation/prediction_probas'):
        os.mkdir(full_path + '/model_evaluation/prediction_probas')
        
    original_headers = pd.read_csv(full_path + "/exploratory/OriginalFeatureNames.csv", sep=',').columns.values.tolist() #Get Original Headers
    for algorithm in algorithms: #loop through algorithms
        print(algorithm)

        for cv_count in range(0,cv_partitions): #loop through cv's
            print(cv_count)
            #Load pickled metric file for given algorithm and cv
            result_file = full_path + '/model_evaluation/pickled_metrics/' + abbrev[algorithm] + "_CV_" + str(cv_count) + "_metrics.pickle"
            file = open(result_file, 'rb')
            results = pickle.load(file)
            file.close()
            
            #Load associated testing dataset
            test_data = pd.read_csv(full_path + '/CVDatasets/'+each+'_CV_' + str(cv_count) + '_Test.csv')
            probas_summary = test_data[[class_label,instance_label]]

            #Separate pickled results
            probas_ = results[9]
            print(probas_[:,1])
            probas_summary['1_prob'] = probas_[:,1]
            file_name = full_path + '/model_evaluation/prediction_probas/' + algorithm + '_CV_'+str(cv_count) + '_case_probas.csv'
            probas_summary.to_csv(file_name, index=False)

Vizualized Datasets: ['hcc-data_example', 'hcc-data_example_custom']
---------------------------------------
hcc-data_example
---------------------------------------
Logistic Regression
0
[0.13909658 0.34753225 0.69530073 0.27665758 0.40951734 0.86868645
 0.14352914 0.86518844 0.51015376 0.81316307 0.80448346 0.07941782
 0.09680545 0.01068842 0.60141133 0.19067556 0.02995119 0.05267555
 0.23074297 0.73666469 0.69896964 0.04716166 0.50473133 0.87931978
 0.05408175 0.06135075 0.07644375 0.66516375 0.01809411 0.02064699
 0.09648349 0.90278128 0.0825265  0.32086796 0.77430599 0.03269612
 0.04812631 0.31910487 0.95228735 0.29694146 0.90826861 0.9360554
 0.53401903 0.94808395 0.54508637 0.07731481 0.0857729  0.16635686
 0.06041972 0.45966884 0.87828541 0.71465484 0.85012105 0.10625736
 0.12936703]
1
[9.63652294e-01 1.65194907e-02 6.34190614e-02 9.99953469e-01
 8.28613308e-04 4.62923481e-02 9.90438872e-01 9.11029868e-01
 6.80938575e-06 9.99999994e-01 3.52653054e-05 9.99978753e-01
 9.40833512e